In [25]:
all_sentences = [
    "ダーウィンの進化論は嘘です",
    "バナナはおやつに入ります",
    "地球は平面です",
    "日本では応用数学科はまだまだ少ない方です",
    "数学と数理科学の違いはなんですか",
    "散歩をするとバナナがおやつに入らないかもしれないことに気づきました",
    "地球の半径を数理的に求めることができます",
    "私はバナナが好きな数学者です",
    "数学とバナナは同じです",
    "残念ながら、地球とバナナとゴリラは同じではありません",   #10
    "ダーウィンはバナナをおやつと考えました",
    "これ以上無意味な文章を作ることをやめませんか",
    "数理の世界は長い年月を経て進歩してきましたが、人間は長い年月を経てゴリラに近づきました",
    "ダーウィンは進化論の提唱者ですが、ダテミキオはカロリー0理論の提唱者です",
    "その理論を応用することで、バナナを用いてブラックホールを生成する方法を数学的に導くことができます",
    "ピザはその高さを0に近づけることで体積が0に近づくためカロリーは0",
    "ダーウィンはゴリラの進化元です",
    "バナナのカロリーは1本86キロカロリーです",
    "どうして地球にはピザが自生していないのですか",
    "ここまでだいたい嘘"
]

In [26]:
from sentence_transformers import SentenceTransformer

# 日本語に強そうなモデルを指定
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# 20行のセンテンスを768次元のベクトルに変換（埋め込み）
all_embeddings = model.encode(all_sentences)
all_embeddings.shape

(20, 768)

In [27]:
from transformers import AutoTokenizer

# 指定したtokenizerを用いる
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

all_tokens = [tokenizer.tokenize(sentence.lower()) for sentence in all_sentences]
all_tokens[0]

['ダーウィン', 'の', '進化', '論', 'は', '嘘', 'です']

In [28]:
# Pineconeの初期化
from pinecone import Pinecone, PodSpec
pc = Pinecone(api_key='1a0e3dd8-12dd-42cc-92ef-16e20a38cb4b')

In [29]:
# インデックスの作成
# all_tokensを格納するDBの作成
pc.create_index(
    name="semantic-search-in-japanese", 
    
    dimension=all_embeddings.shape[1], 
    metric="cosine",
    spec=PodSpec(
        environment='gcp-starter', 
        pod_type='starter'
    )
)

In [30]:
# インデックスに接続
index = pc.Index(name="semantic-search-in-japanese")

In [31]:
# all_embeddingsとall_tokensをInsertする

# all_embeddings...単語のベクトル埋め込み（数値）
# all_tokens...単語のベクトル
upserts = []
for i, (embedding, tokens) in enumerate(zip(all_embeddings, all_tokens)):
    upserts.append((str(i), embedding.tolist(), {'tokens': tokens}))

# 20行の文章が20個のベクトルとして格納される
index.upsert(vectors=upserts)

{'upserted_count': 20}

In [33]:
# セマンティック検索
query = "ゴリラ"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '19',
              'metadata': {'tokens': ['ここ', 'まで', 'だい', '##たい', '嘘']},
              'score': 0.527227104,
              'values': []},
             {'id': '1',
              'metadata': {'tokens': ['バナナ', 'は', 'お', 'やつ', 'に', '入り', 'ます']},
              'score': 0.415789366,
              'values': []},
             {'id': '11',
              'metadata': {'tokens': ['これ',
                                      '以上',
                                      '無意',
                                      '##味',
                                      'な',
                                      '文章',
                                      'を',
                                      '作る',
                                      'こと',
                                      'を',
                                      'やめ',
                                      'ませ',
                                      'ん',
                                      'か']},
              'score': 0.371

In [49]:
# 検索結果が怪しいので別のtokenizerを使う
from transformers import BertJapaneseTokenizer

# 指定したtokenizerを用いる
tokenizer_cl_tohoku = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

all_tokens_cl_tohoku = [tokenizer_cl_tohoku.tokenize(sentence) for sentence in all_sentences]
all_tokens_cl_tohoku[0]

['ダーウィン', 'の', '進化', '論', 'は', '嘘', 'です']

In [36]:
# 以前のベクトルを削除し、新しいベクトルを格納
index.delete(delete_all=True)

{}

In [50]:
upserts = []
for i, (embedding, tokens) in enumerate(zip(all_embeddings, all_tokens_cl_tohoku)):
    upserts.append((str(i), embedding.tolist(), {'tokens': tokens}))

index.upsert(vectors=upserts)

{'upserted_count': 20}

In [51]:
# セマンティック検索
query = "ゴリラはバナナを食べましたか？"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '1',
              'metadata': {'tokens': ['バナナ', 'は', 'お', 'やつ', 'に', '入り', 'ます']},
              'score': 0.804515719,
              'values': []},
             {'id': '10',
              'metadata': {'tokens': ['ダーウィン',
                                      'は',
                                      'バナナ',
                                      'を',
                                      'お',
                                      'やつ',
                                      'と',
                                      '考え',
                                      'まし',
                                      'た']},
              'score': 0.73956424,
              'values': []},
             {'id': '5',
              'metadata': {'tokens': ['散歩',
                                      'を',
                                      'する',
                                      'と',
                                      'バナナ',
                                      'が',
            

In [48]:
# セマンティック検索
query = "ゴリラ"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '19',
              'metadata': {'tokens': ['ここ', 'まで', 'だい', '##たい', '嘘']},
              'score': 0.527227104,
              'values': []},
             {'id': '1',
              'metadata': {'tokens': ['バナナ', 'は', 'お', 'やつ', 'に', '入り', 'ます']},
              'score': 0.415789366,
              'values': []},
             {'id': '11',
              'metadata': {'tokens': ['これ',
                                      '以上',
                                      '無意',
                                      '##味',
                                      'な',
                                      '文章',
                                      'を',
                                      '作る',
                                      'こと',
                                      'を',
                                      'やめ',
                                      'ませ',
                                      'ん',
                                      'か']},
              'score': 0.371

In [53]:
pc.delete_index("semantic-search-in-japanese")